# Yet Another ProPublica COMPAS notebook

## Data loading and pre-processing

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt

!pip install fairlearn 

df = pd.read_csv('https://raw.githubusercontent.com/javism/seminariofate2025/main/data/propublica_recidivism_train.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/javism/seminariofate2025/main/data/propublica_recidivism_test.csv')

In [ ]:
df

We plot the histogram of the scores by "race".

In [ ]:
df['race_label'] = 'black'
df.loc[df['race'] == 0, 'race_label'] = 'white'
df_test['race_label'] = 'black'
df_test.loc[df_test['race'] == 0, 'race_label'] = 'white'

df['decile_score'].hist(by=df['race_label'], figsize = (8,3))
df['y'].hist(by=df['race_label'], figsize = (8,3))

In [ ]:
def z_score_standardization(series):
    return (series - series.mean()) / series.std()

df_s = df.copy()
for col in df_s.columns:
    if col != 'race_label':
        df_s[col] = z_score_standardization(df_s[col])
    
df_s.groupby('race_label').boxplot(rot=70)

## Preprocessing


In [ ]:
# Remove some variables, standarize and binarize label
race_label = df['race_label']
race_label_t = df_test['race_label']
df.drop(['c_charge_desc','decile_score','score_text', 'race_label'], axis=1, inplace=True)
df_test.drop(['c_charge_desc','decile_score','score_text', 'race_label'], axis=1, inplace=True)

X = df.drop(['y'], axis=1)
feature_names = X.columns
y = df['y']

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
X_test = df_test.drop(['y'], axis=1)
y_test = df_test['y']
X_test = scaler.transform(X_test)

## Disparity metrics auxiliary functions

In [ ]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from fairlearn.metrics import false_positive_rate
from fairlearn.metrics import false_negative_rate
from fairlearn.metrics import true_positive_rate
from fairlearn.metrics import selection_rate
from fairlearn.metrics import count

def performance_metrics(y_test, predictions, race_label_t): 
    # Test for discrimination in predictions
    from fairlearn.metrics import MetricFrame,false_negative_rate,false_positive_rate
    from sklearn.metrics import accuracy_score

    grouped_metric = MetricFrame(metrics=accuracy_score,y_true=y_test,y_pred=predictions, sensitive_features=race_label_t)
    print(f"Overall Accuracy = \t{grouped_metric.overall:.4f}")
    print("Accuracy by groups:")
    for key, value in grouped_metric.by_group.to_dict().items():
        print(f"Acc {key}: \t\t{value:.4f}")

    grouped_metric = MetricFrame(metrics=false_negative_rate,y_true=y_test,y_pred=predictions, sensitive_features=race_label_t)
    print(f"Overall FNR = \t\t{grouped_metric.overall:.4f}")
    print("FNR by groups: ")
    for key, value in grouped_metric.by_group.to_dict().items():
        print(f"FNR {key}: \t\t{value:.4f}")

    grouped_metric = MetricFrame(metrics=false_positive_rate,y_true=y_test,y_pred=predictions, sensitive_features=race_label_t)
    print(f"Overall FPR = \t\t{grouped_metric.overall:.4f}")
    print("FPR by groups: ")
    for key, value in grouped_metric.by_group.to_dict().items():
        print(f"\x1B[1mFPR {key}: \t\t{value:.4f}")
        

def performance_plots(y_test, predictions, race_label_t): 
    metrics = {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'false positive rate': false_positive_rate,
        'false negative rate': false_negative_rate,
        'count': count}
    metric_frame = MetricFrame(metrics=metrics,
                               y_true=y_test,
                               y_pred=predictions,
                               sensitive_features=race_label_t)
    metric_frame.by_group.plot.bar(
        subplots=True,
        layout=[3, 3],
        legend=False,
        figsize=[12, 8],
        title="LR",
    )
    plt.show()


## Basic model fitting and performance evaluation

<div class="alert alert-block alert-info">
<b>Question:</b> What conclusions do you draw from the performance by groups?
</div>

In [ ]:
# Fit a logistic regression predictor
lr = LogisticRegression()
lr.fit(X,y)

from sklearn.metrics import confusion_matrix
lr.score(X_test, y_test)
predictions = lr.predict(X_test)

performance_metrics(y_test, predictions, race_label_t)

In [ ]:
performance_plots(y_test, predictions, race_label_t)


## Print the linear part of our mathematical model

<img src="pics/compas_diagram_pre.png" alt="COMPAS model as a graph" class="bg-primary" width="500px">

In [ ]:
intercept = lr.intercept_[0];
print(f'{intercept:.3f} ', end="")
for c, f in zip(lr.coef_.ravel(), list(feature_names) ):
    print(f'{c:.3f} * {f}', end=' + ')

# Feature Selection

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
print(X.shape)

lsvc = LinearSVC(C=0.001, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X = model.transform(X)
X_test = model.transform(X_test)
print(X.shape)
print(X_test.shape)

print(f"Features selected by SelectFromModel: {feature_names[model.get_support()]}")

## Fitting the model with feature selection

In [ ]:
# Fit a logistic regression predictor
lr = LogisticRegression()
lr.fit(X,y)

from sklearn.metrics import confusion_matrix
lr.score(X_test, y_test)
predictions = lr.predict(X_test)

performance_metrics(y_test, predictions, race_label_t)
performance_plots(y_test, predictions, race_label_t)

## Print the linear part of our mathematical model

In [ ]:
intercept = lr.intercept_[0];
print(f'{intercept:.3f} ', end="")
for c, f in zip(lr.coef_.ravel(), list(feature_names[model.get_support()]) ):
    print(f'{c:.3f} * {f}', end=' + ')

<div class="alert alert-block alert-info">
<b>Question:</b> What conclusions do you draw from the pruned model?
</div>

## Final model

This mathematical formula is equivalent to the NorthPointe model which, based on 137 variables, predicts whether a person will reoffend. 

<img src="pics/compas_diagram.png" alt="COMPAS model as a graph" class="bg-primary" width="500px">

# Conclusions

* Any model built on historical data is going to reproduce structural inequalities.

* Even by minimising the variables they are mediated (causal inference term) by "race".

* Would you buy an intelligent tool that predicts behaviour from 137 variables, what if it has 2 variables?

* There are other more important underlying questions about what we mean by justice and the impossibility of implementing it algorithmically if we consider that any AI tool does not incorporate context into its decisions.

# References
* Angwin, J., & Larson, J. (2016, diciembre 30). Bias in Criminal Risk Scores Is Mathematically Inevitable, Researchers Say. ProPublica. <https://www.propublica.org/article/bias-in-criminal-risk-scores-is-mathematically-inevitable-researchers-say>
* Larson, J., & Angwin, J. (2016, mayo 23). How We Analyzed the COMPAS Recidivism Algorithm. ProPublica. <https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm>
* Rudin, C. Stop explaining black box machine learning models for high stakes decisions and use interpretable models instead. Nat Mach Intell 1, 206–215 (2019). <https://doi.org/10.1038/s42256-019-0048-x>